# Estimation 

In this notebook, we construct a giant likelihood with price, viewership, and sampling weights and estimate everything out. 

In [1]:
import ipystata
import os
os.getcwd()

'C:\\Users\\mjbaker\\Documents\\GitHub\\NewsHour'

In [2]:
%%stata -s nh1
clear all
cd C:\Users\mjbaker\Documents\Github\NewsHour
use AveragedDataDyno.dta
set more off
set seed 8675309
sort market stationid mt timeslot


C:\Users\mjbaker\Documents\Github\NewsHour


## Step 1 - read in some data and organize it.

Here, we have to organize data and get everything going. Really, this is the same stuff we did in notebook one to get dynamic data aligned with the (observed) viewership likelihood. ALong these lines, we need to reconstitute a few variables that we use and get them into mata. We will create a marker variable so that things work. 

While previously we developed a "game" variable in `mata`, this time around I'll do it in `Stata` just to see if we get the same result:

In [3]:
%%stata -s nh1
bysort stationid: egen countl = total(lnews)
bysort stationid: gen lasts = _n == _N

gen game=lnpps!=. & (lnews | otherl)
replace game = 0 if countl ==0


(1,328 real changes made)


In [5]:
%%stata -s nh1
tab game if timeslot == 8


       game |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |     16,222       97.59       97.59
          1 |        400        2.41      100.00
------------+-----------------------------------
      Total |     16,622      100.00


In [6]:
%%stata -s nh1
gen useOrd =_n
tsset stationid timeslot

gen otherlPrime=lnews&game
gen lnewsPrime =otherl&game
gen nnewsPrime =nnews&game

gen lnewslnewsPrime=otherl*l.lnews
gen lnewsnnewsPrime=otherl*l.nnews
gen nnewslnewsPrime=nnews*l.lnews
gen nnewsnnewsPrime=nnews*l.nnews
gen lsiPrime=l.si
replace lsiPrime=ln(lsiPrime)
replace lsiPrime=0 if lsiPrime==.

gen siXlnlnPrime=lnewslnewsPrime*lsi
gen siXlnnnPrime=lnewsnnewsPrime*lsi
gen siXnnlnPrime=nnewslnewsPrime*lsi
gen siXnnnnPrime=nnewsnnewsPrime*lsi
gen lnewstotPrime=otherl*ln(1+totslnews)
gen nnewstotPrime=nnews*ln(1+totsnnews)
sort useOrd 


       panel variable:  stationid (strongly balanced)
        time variable:  timeslot, 3 to 8
                delta:  1 unit
(16,622 missing values generated)
(16,622 missing values generated)
(16,622 missing values generated)
(16,622 missing values generated)
(16,622 missing values generated)
(83,110 real changes made)
(16,622 real changes made)
(16,622 missing values generated)
(16,622 missing values generated)
(16,622 missing values generated)
(16,622 missing values generated)


Pull all the variables into `mata`:

In [7]:
%%stata -s nh1
mata:
    st_view(lnewsPrime=.,.,"lnewsPrime")
    st_view(otherlPrime=.,.,"otherlPrime")
    st_view(nnewsPrime=.,.,"nnewsPrime")
    st_view(lnewslnewsPrime=.,.,"lnewslnewsPrime")
    st_view(lnewsnnewsPrime=.,.,"lnewsnnewsPrime")
    st_view(nnewslnewsPrime=.,.,"nnewslnewsPrime")
    st_view(nnewsnnewsPrime=.,.,"nnewsnnewsPrime")
    st_view(lsiPrime=.,.,"lsiPrime")
    st_view(siXlnlnPrime=.,.,"siXlnlnPrime")
    st_view(siXlnnnPrime=.,.,"siXlnnnPrime")
    st_view(siXnnlnPrime=.,.,"siXnnlnPrime")
    st_view(siXnnnnPrime=.,.,"siXnnnnPrime")
    st_view(lnewstotPrime=.,.,"lnewstotPrime")
    st_view(nnewstotPrime=.,.,"nnewstotPrime")

    st_view(lnewslnews=.,.,"lnewslnews")
    st_view(lnewsnnews=.,.,"lnewsnnews")
    st_view(nnewslnews=.,.,"nnewslnews")
    st_view(nnewsnnews=.,.,"nnewsnnews")
    st_view(lsi=.,.,"lsi")
    st_view(siXlnln=.,.,"siXlnln")
    st_view(siXlnnn=.,.,"siXlnnn")
    st_view(siXnnln=.,.,"siXnnln")
    st_view(siXnnnn=.,.,"siXnnnn")
    st_view(lnewstot=.,.,"lnewstot")
    st_view(nnewstot=.,.,"nnewstot")
    st_view(l_ACS_HH=.,.,"l_ACS_HH")
    st_view(nnewsn=.,.,"nnewsn")
    st_view(lnewsn=.,.,"lnewsn")
    st_view(otherln=.,.,"otherln")
    st_view(othercn=.,.,"othercn")
end


:     st_view(lnewsPrime=.,.,"lnewsPrime")

:     st_view(otherlPrime=.,.,"otherlPrime")

:     st_view(nnewsPrime=.,.,"nnewsPrime")

:     st_view(lnewslnewsPrime=.,.,"lnewslnewsPrime")

:     st_view(lnewsnnewsPrime=.,.,"lnewsnnewsPrime")

:     st_view(nnewslnewsPrime=.,.,"nnewslnewsPrime")

:     st_view(nnewsnnewsPrime=.,.,"nnewsnnewsPrime")

:     st_view(lsiPrime=.,.,"lsiPrime")

:     st_view(siXlnlnPrime=.,.,"siXlnlnPrime")

:     st_view(siXlnnnPrime=.,.,"siXlnnnPrime")

:     st_view(siXnnlnPrime=.,.,"siXnnlnPrime")

:     st_view(siXnnnnPrime=.,.,"siXnnnnPrime")

:     st_view(lnewstotPrime=.,.,"lnewstotPrime")

:     st_view(nnewstotPrime=.,.,"nnewstotPrime")

:     st_view(lnewsnnews=.,.,"lnewsnnews")

:     st_view(nnewslnews=.,.,"nnewslnews")

:     st_view(nnewsnnews=.,.,"nnewsnnews")

:     st_view(lsi=.,.,"lsi")

:     st_view(siXlnln=.,.,"siXlnln")

:     st_view(siXlnnn=.,.,"siXlnnn")

:     st_view(siXnnln=.,.,"siXnnln")

:     st_view(siXnnnn=.,.,"siXnnnn")

:  

To make it easier to pass things around, let's create a structured object and use it to pass all this stuff around. We also make a simple initiation function to deal with it (this would be a bit simpler in Python!)

In [8]:
%%stata -s nh1
mata: 
    struct dynoInfo {
        real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews, lsi,
            siXlnln, siXlnnn, siXnnln, siXnnnn, lnewstot, nnewstot, l_ACS_HH,
            lnewsn,otherln,nnewsn,othercn
    }
    
    struct dynoInfo dynoInfoInit(real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
        lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,lnewsn,otherln,nnewsn,othercn)
    {
        struct dynoInfo scalar Dy
    
        Dy.lnewslnews=lnewslnews
        Dy.lnewsnnews=lnewsnnews
        Dy.nnewslnews=nnewslnews
        Dy.nnewsnnews=nnewsnnews
        Dy.lsi=lsi
        Dy.siXlnln=siXlnln
        Dy.siXlnnn=siXlnnn
        Dy.siXnnln=siXnnln
        Dy.siXnnnn=siXnnnn
        Dy.lnewstot=lnewstot
        Dy.nnewstot=nnewstot
        Dy.l_ACS_HH=l_ACS_HH
        Dy.lnewsn=lnewsn
        Dy.otherln=otherln
        Dy.nnewsn=nnewsn
        Dy.othercn=othercn
        return(Dy)
    }
end


:     struct dynoInfo {
>         real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews, lsi,
>             siXlnln, siXlnnn, siXnnln, siXnnnn, lnewstot, nnewstot, l_ACS_HH,
>             lnewsn,otherln,nnewsn,othercn
>     }

:     
:     struct dynoInfo dynoInfoInit(real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
>         lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,lnewsn,otherln,nnewsn,othercn)
>     {
>         struct dynoInfo scalar Dy
>     
>         Dy.lnewslnews=lnewslnews
>         Dy.lnewsnnews=lnewsnnews
>         Dy.nnewslnews=nnewslnews
>         Dy.nnewsnnews=nnewsnnews
>         Dy.lsi=lsi
>         Dy.siXlnln=siXlnln
>         Dy.siXlnnn=siXlnnn
>         Dy.siXnnln=siXnnln
>         Dy.siXnnnn=siXnnnn
>         Dy.lnewstot=lnewstot
>         Dy.nnewstot=nnewstot
>         Dy.l_ACS_HH=l_ACS_HH
>         Dy.lnewsn=lnewsn
>         Dy.otherln=otherln
>         Dy.nnewsn=nnewsn
>         Dy.othercn=othercn
>         return(Dy)
>  

Now, just pass all the information along for use in estimation.

In [9]:
%%stata -s nh1
mata:
    Dy=dynoInfoInit(lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
        lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,
        lnewsn,otherln,nnewsn,othercn)
end


:     Dy=dynoInfoInit(lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
>         lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,
>         lnewsn,otherln,nnewsn,othercn)


## Price information

Since the above is a pretty convenient way of dealing with a lot of variables, let's also use our previously constructed and saved data on pricing, and pack it up in a similar way:

In [10]:
%%stata -s nh1
mata:
    mata matuse MataPriceData
end
    


:     mata matuse MataPriceData
(loading idt[3738,1], l_ACS_HHt[3738,1], lnewst[3738,1], lnppst[3738,1], lnviewt[3738,1], mat[101,2], mtt[3738,1], nnewst[3738,1],
 otherlt[3738,1])


A structured object containing all this information, and using it to pass stuff along:

In [11]:
%%stata -s nh1
mata:
   struct priceInfo {
        real matrix y,lnewst,otherlt,nnewst,lnviewt,l_ACS_HHt,id,mt,m
    }
    struct priceInfo priceInfoInit(y,lnewst,otherlt,nnewst,lnviewt,l_ACS_HHt,id,mt,m)
    {
        struct priceInfo scalar pInfo
    
        pInfo.y=y
        pInfo.lnewst=lnewst
        pInfo.otherlt=otherlt
        pInfo.nnewst=nnewst
        pInfo.lnviewt=lnviewt
        pInfo.id=id
        pInfo.mt=mt
        pInfo.m=m
        pInfo.l_ACS_HHt=l_ACS_HHt
    return(pInfo)
    }
end


:    struct priceInfo {
>         real matrix y,lnewst,otherlt,nnewst,lnviewt,l_ACS_HHt,id,mt,m
>     }

:     struct priceInfo priceInfoInit(y,lnewst,otherlt,nnewst,lnviewt,l_ACS_HHt,id,mt,m)
>     {
>         struct priceInfo scalar pInfo
>     
>         pInfo.y=y
>         pInfo.lnewst=lnewst
>         pInfo.otherlt=otherlt
>         pInfo.nnewst=nnewst
>         pInfo.lnviewt=lnviewt
>         pInfo.id=id
>         pInfo.mt=mt
>         pInfo.m=m
>         pInfo.l_ACS_HHt=l_ACS_HHt
>     return(pInfo)
>     }


Import our information on selection stuff and the game and all that:

In [12]:
%%stata -s nh1
mata:
    mata matuse gsAndus, replace
end


:     mata matuse gsAndus, replace
(loading NashW[99732,20], Up[99732,20], Upb[99732,20], Upg[99732,20], Upre1[99732,20], Upre1g[99732,20], Upre2[99732,20], Upre2g[99732,20],
 Uv[99732,20], Uvg[99732,20], Uvre1[99732,20], Uvre1g[99732,20], Uvre2[99732,20], Uvre2g[99732,20], Uvsg[99732,20], Uvsi[99732,20],
 id[99732,1], market[99732,1], mt[99732,1], weights1[99732,1], weights2[99732,1])


Just to e careful...make sure we have the indicators in the right order:

In [14]:
%%stata -s nh1 --mata
    ord = order((market, id, mt), 1..3)
    


Mata output:

:     ord = order((market, id, mt), 1..3)

:     


## Passing Along the Ackerberg/Game Information

We now have to make an efficient way of passing information to the likelihood function. This will include all of our weights, along with all of our error bounds, etc. So, here is the structured object (which we probably should have broken up into smaller pieces!)

** Note:** the weights 1 and weights 2 variables pertain to random effects. Otherwise, anything with a following `g` is an importance sampling weight. 

In [15]:
%%stata -s nh1 --mata
    struct gsAndws {
        real matrix NashW, Up, Upb, Upg, Uv, Uvb, Uvg,
            Uvsi, Uvsg,
            lnewsPrime, otherlPrime, nnewsPrime,
            lnewslnewsPrime, lnewsnnewsPrime, nnewslnewsPrime,
            nnewsnnewsPrime, lsiPrime, siXlnlnPrime, siXlnnnPrime,
            siXnnlnPrime, siXnnnnPrime, lnewstotPrime, nnewstotPrime,
            Uvre1, Uvre2, Upre1, Upre2, 
            weights1, weights2, Uvre1g, Uvre2g, Upre1g, Upre2g, lnview, l_ACS_HH
    }


Mata output:

:     struct gsAndws {
>         real matrix NashW, Up, Upb, Upg, Uv, Uvb, Uvg,
>             Uvsi, Uvsg,
>             lnewsPrime, otherlPrime, nnewsPrime,
>             lnewslnewsPrime, lnewsnnewsPrime, nnewslnewsPrime,
>             nnewsnnewsPrime, lsiPrime, siXlnlnPrime, siXlnnnPrime,
>             siXnnlnPrime, siXnnnnPrime, lnewstotPrime, nnewstotPrime,
>             Uvre1, Uvre2, Upre1, Upre2, 
>             weights1, weights2, Uvre1g, Uvre2g, Upre1g, Upre2g, lnview, l_ACS_HH
>     }


Here we initialize this structure. Note that some amount of care must be taken so that everything matches...but dont' forget that this is really just a delivery system!

In [19]:
%%stata -s nh1 --mata 
    struct gsAndws gsAndwsInit(NashW, Up, Upb, Upg, Uv ,Uvb, Uvg,
        Uvsi,Uvsg,
        lnewsPrime, otherlPrime, nnewsPrime,
        lnewslnewsPrime, lnewsnnewsPrime, nnewslnewsPrime, 
        nnewsnnewsPrime, lsiPrime, siXlnlnPrime,
        siXlnnnPrime, siXnnlnPrime, siXnnnnPrime, lnewstotPrime, nnewstotPrime,
        Uvre1, Uvre2, Upre1, Upre2, 
        weights1, weights2, Uvre1g, Uvre2g, Upre1g, Upre2g, lnview, l_ACS_HH)
    {
        struct gsAndws scalar GW

        GW.NashW=NashW
        GW.Up    = Up
        GW.Upb   = Upb
        GW.Upg   = Upg
        GW.Uv    = Uv
        GW.Uvb   = Uvb
        GW.Uvg   = Uvg
        GW.Uvsi  = Uvsi
        GW.Uvsg  = Uvsg
        
        GW.lnewsPrime      = lnewsPrime
        GW.otherlPrime     = otherlPrime
        GW.nnewsPrime      = nnewsPrime
        GW.lnewslnewsPrime = lnewslnewsPrime
        GW.lnewsnnewsPrime = lnewsnnewsPrime
        GW.nnewslnewsPrime = nnewslnewsPrime
        GW.nnewsnnewsPrime = nnewsnnewsPrime
        GW.lsiPrime        = lsiPrime
        GW.siXlnlnPrime    = siXlnlnPrime
        GW.siXlnnnPrime    = siXlnnnPrime
        GW.siXnnlnPrime    = siXnnlnPrime
        GW.siXnnnnPrime    = siXnnnnPrime
        GW.lnewstotPrime   = lnewstotPrime
        GW.nnewstotPrime   = nnewstotPrime
        GW.Uvre1    = Uvre1
        GW.Uvre2    = Uvre2
        GW.Upre1    = Upre1
        GW.Upre2    = Upre2
        GW.weights1 = weights1
        GW.weights2 = weights2
        GW.Uvre1g   = Uvre1g
        GW.Uvre2g   = Uvre2g
        GW.Upre1g   = Upre1g
        GW.Upre2g   = Upre2g
        GW.lnview   = lnview
        GW.l_ACS_HH = l_ACS_HH

        return(GW)
    }

Mata output:

:     struct gsAndws gsAndwsInit(NashW, Up, Upb, Upg, Uv ,Uvb, Uvg,
>         Uvsi,Uvsg,
>         lnewsPrime, otherlPrime, nnewsPrime,
>         lnewslnewsPrime, lnewsnnewsPrime, nnewslnewsPrime, 
>         nnewsnnewsPrime, lsiPrime, siXlnlnPrime,
>         siXlnnnPrime, siXnnlnPrime, siXnnnnPrime, lnewstotPrime, nnewstotPrime,
>         Uvre1, Uvre2, Upre1, Upre2, 
>         weights1, weights2, Uvre1g, Uvre2g, Upre1g, Upre2g, lnview, l_ACS_HH)
>     {
>         struct gsAndws scalar GW
> 
>         GW.NashW=NashW
>         GW.Up    = Up
>         GW.Upb   = Upb
>         GW.Upg   = Upg
>         GW.Uv    = Uv
>         GW.Uvb   = Uvb
>         GW.Uvg   = Uvg
>         GW.Uvsi  = Uvsi
>         GW.Uvsg  = Uvsg
>         
>         GW.lnewsPrime      = lnewsPrime
>         GW.otherlPrime     = otherlPrime
>         GW.nnewsPrime      = nnewsPrime
>         GW.lnewslnewsPrime = lnewslnewsPrime
>         GW.lnewsnnewsPrime = lnewsnnewsPrime
>         GW.nnewslnewsPrime = nn

In [18]:
%%stata -s nh1 --mata
mata drop gsAndwsInit()

Mata output:




Now, we pare all these down so that we just have the stuff for those involved in the game, (I checked to make sure things ling up as well). So:

In [20]:
%%stata -s nh1 --mata
    NashW = select(NashW, game)
        _editmissing(NashW,  0)
    Up     = select(Up,   game)
    Upb    = select(Upb,  game)
    Upg    = select(Upg,  game)
    Uvsi   = select(Uvsi, game)
    Uvsg   = select(Uvsg, game)
    Uv     = select(Uv,   game)
    Uvb    = select(Uvb,  game)
    Uvg    = select(Uvg,  game)
    
    Uvre1  =   select(Uvre1, game)
    Uvre2  =   select(Uvre2, game)
    Upre1  =   select(Upre1, game)
    Upre2  =    select(Upre2, game)
    weights1  = select(weights1, game)
    weights2  = select(weights2, game)
    Uvre1g    = select(Uvre1g, game)
    Uvre2g    = select(Uvre2g, game)
    Upre1g    = select(Upre1g, game)
    Upre2g    = select(Upre2g, game)
    
    Mpop=select(Mpop,game)

    lnewsPrime  = select(lnewsPrime, game)
    nnewsPrime  = select(nnewsPrime, game)
    otherlPrime = select(otherlPrime,game)
    lnewslnewsPrime  = select(lnewslnewsPrime, game)
        _editmissing(lnewslnewsPrime, 0)
    lnewsnnewsPrime  = select(lnewsnnewsPrime, game)
        _editmissing(lnewsnnewsPrime, 0)
    nnewslnewsPrime  = select(nnewslnewsPrime,game)
        _editmissing(nnewslnewsPrime,0)
    nnewsnnewsPrime  = select(nnewsnnewsPrime,game)
        _editmissing(nnewsnnewsPrime,0)
    lsiPrime      =  select(lsiPrime,game)
        _editmissing(lsiPrime,0)
    siXlnlnPrime  =  select(siXlnlnPrime,game)
        _editmissing(siXlnlnPrime,0)
    siXlnnnPrime  =  select(siXlnnnPrime,game)
        _editmissing(siXlnnnPrime,0)
    siXnnlnPrime  =  select(siXnnlnPrime,game)
        _editmissing(siXnnlnPrime,0)
    siXnnnnPrime  =  select(siXnnnnPrime,game)
        _editmissing(siXnnnnPrime,0)
    lnewstotPrime  = select(lnewstotPrime,game)
    nnewstotPrime  = select(nnewstotPrime,game)
    
    l_ACS_HHPrime  = select(l_ACS_HH,game)

Mata output:

:     NashW = select(NashW, game)

:         _editmissing(NashW,  0)

:     Up     = select(Up,   game)

:     Upb    = select(Upb,  game)

:     Upg    = select(Upg,  game)

:     Uvsi   = select(Uvsi, game)

:     Uvsg   = select(Uvsg, game)

:     Uv     = select(Uv,   game)

:     Uvb    = select(Uvb,  game)
                 <istmt>:  3499  Uvb not found
-------------------------------------------------------------------------------------------------------------------------------------------------
r(3499);
command Uvg is unrecognized
r(199);
.     Uvre1  =   select(Uvre1, game)
command Uvre1 is unrecognized
r(199);
command Uvre2 is unrecognized
r(199);
command Upre1 is unrecognized
r(199);
command Upre2 is unrecognized
r(199);
command weights1 is unrecognized
r(199);
command weights2 is unrecognized
r(199);
command Uvre1g is unrecognized
r(199);
command Uvre2g is unrecognized
r(199);
command Upre1g is unrecognized
r(199);
command Upre2g is unrecognized
r(199);
.     M

Next we can pass all of this to the structured object - or more correctly, the function that initiates it. Here goes